This notebook tests which pages from SWDE are available in archive.org. We could then use those because they have also their CSS archived.

In [ ]:
import importlib

from awe.data import swde, wayback
for module in [wayback]:
    importlib.reload(module)

In [ ]:
# Try only subset for now.
vertical = swde.VERTICALS[0]
website = vertical.websites[0]
pages = website.pages[:10]
page = pages[0]

In [ ]:
SWDE_TIMESTAMP = '20110601000000' # SWDE was released in 2011

In [ ]:
wayback.WaybackPage.get(page.url, SWDE_TIMESTAMP)